In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
import splat
import astropy.units as u
import popsims
#splat.initializeStandards(all=True)
from scipy import interpolate
from tqdm import tqdm
from astropy.table import Table
#import wisps
%matplotlib inline

/users/caganze/research/splat


Welcome to the Spex Prism Library Analysis Toolkit (SPLAT)!
If you make use of any features of this toolkit for your research, please remember to cite the SPLAT paper:

Burgasser et al. (2017, Astro. Soc. India Conf. Series 14, p. 7); Bibcode: 2017ASInC..14....7B

If you make use of any spectra or models in this toolkit, please remember to cite the original source.
Please report any errors are feature requests to our github page, https://github.com/aburgasser/splat/




In [2]:
#read in Zhang samples
files=glob.glob('/volumes/LaCie/zhang/*/*.fits')

In [3]:
len(files)

121

In [4]:
def get_spectrum(f):
    try:
        hdu=fits.open(f)[0]
        wave=np.array([hdu.header['CRVAL1']+hdu.header['CDELT1']*x for x \
          in range(0, hdu.header['NAXIS1'])])*u.angstrom.to(u.micron)
        flux= hdu.data*u.erg/(u.cm**2 * u.s * u.micron)
        s=splat.Spectrum(wave=wave, flux=flux, noise=flux/1000, name=f.split('/')[-1])
        #s.smooth(150)
        #s.toInstrument('SpeX')
        #plt.plot(s.wave, s.flux)
        return s
    except:
        print ('could not read file {}'.format(f))
        return None

In [5]:
def interpolated_standards():
    stds=splat.STDS_DWARF_SPEX
    interpstds={}
    for k in stds.keys():
        s=stds[k]
        print (k)
        s.normalize()
        s.toInstrument('WFC3-G141')
        interpstds[k]=(interpolate.interp1d(s.wave.value, s.flux.value), interpolate.interp1d(s.wave.value, s.noise.value))
    return interpstds

In [6]:
spectra=[ get_spectrum(f) for f in tqdm(files)]

 16%|██████▌                                   | 19/121 [00:05<00:26,  3.81it/s]

could not read file /volumes/LaCie/zhang/spectra/ULAS_J002136.00+155227.3_T4p_OSIRIS_Primeval-VI.fits
could not read file /volumes/LaCie/zhang/spectra/ULAS_J000844.34+012729.4_T6.5_OSIRIS_Primeval-VI.fits


 20%|████████▎                                 | 24/121 [00:06<00:20,  4.72it/s]

could not read file /volumes/LaCie/zhang/spectra/ULAS_J012947.35+151143.1_T4_OSIRIS_Primeval-VI..fits


 39%|████████████████▎                         | 47/121 [00:09<00:06, 12.31it/s]

...

 50%|█████████████████████▏                    | 61/121 [00:10<00:04, 14.33it/s]

could not read file /volumes/LaCie/zhang/spectra/ULAS_J130710.22+151103.4_sdL8_Xshooter_NIR_Primeval-IV.fits


 64%|██████████████████████████▋               | 77/121 [00:11<00:01, 25.35it/s]

could not read file /volumes/LaCie/zhang/spectra/ULAS_J135058.85+081506.8_usdL3_Xshooter_NIR_Primeval-III.fits
could not read file /volumes/LaCie/zhang/spectra/ULAS_J135058.85+081506.8_usdL3_Xshooter_VIS_Primeval-III.fits


100%|█████████████████████████████████████████| 121/121 [00:15<00:00,  7.94it/s]


In [7]:
spectra

[ spectrum of ULAS_J000509.42+101407.7_T3_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J002136.00+155227.3_T4p_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J105131.80+025751.0_T7_X-shooter_NIR_Primeval-VI.fits,
  spectrum of ULAS_J004030.41+091524.8_T4.5_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J014443.26+014741.0_T5_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J124639.32+032314.2_T5_X-shooter_NIR_Primeval-VI.fits,
  spectrum of ULAS_J012252.58+043735.3_T2.5_X-shooter_NIR_Primeval-VI.fits,
  spectrum of ULAS_J112059.40+121904.4_T4.5_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J135950.14+062553.9_T3_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J000844.34+012729.4_T6.5_X-shooter_NIR_Primeval-VI.fits,
  spectrum of ULAS_J150457.65+053800.8_sdT5.5_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J083346.88+001248.0_T3_X-shooter_VIS_Primeval-VI.fits,
  spectrum of ULAS_J004030.41+091524.8_T4.5_X-shooter_NIR_Primeval-VI.fits,
  spectrum of ULAS_J014

In [8]:
spectra=[s for s in spectra if s is not None]
sp_designations=[s.name.split('_')[1] for s in spectra]
sp_types=[s.name.split('_')[2] for s in spectra]


In [9]:
#len(data)

In [24]:
#get photometry information
phot=Table(fits.open('/volumes/LaCie/zhang/properties.fit')[1].data)

In [26]:
phot=Table(fits.open('/volumes/LaCie/zhang/parallaxes.fit')[1].data)

In [28]:
phot.to_pandas()

,recno,SName,pmRA,e_pmRA,pmDE,e_pmDE,Dist,E_Dist,e_dist_lc,Vtan,E_Vtan,e_vtan_lc,Name
0,1,UL0118+03,14,12,-80,12,209,20,18,80,18,18,ULAS J011824.89+034130.4\r
1,2,UL0212+06,2,4,-429,5,139,13,12,282,27,25,ULAS J021258.08+064115.9\r
2,3,UL0238+05,186,18,-145,15,95,9,8,106,15,14,ULAS J023803.12+054526.1\r
3,4,UL0753+20,-36,1,-191,1,74,7,6,68,7,6,ULAS J075335.23+200622.4\r
4,5,UL0822+04,35,7,-154,5,92,9,8,69,8,7,ULAS J082206.61+044101.8\r
5,6,UL1114+07,-17,9,-306,6,211,20,18,306,31,29,ULAS J111429.54+072809.5\r
6,7,UL1231+01,-225,6,22,4,164,16,14,176,18,16,ULAS J123142.99+015045.4\r
7,8,UL1241-00,-61,9,-45,5,245,23,21,89,15,14,ULAS J124104.75-000531.4\r
8,9,UL1252+09,-299,7,6,6,117,11,10,166,17,15,ULAS J125226.62+092920.1\r
9,10,UL1307+15,-391,14,-124,13,86,8,8,168,18,17,ULAS J130710.22+151103.4\r


In [11]:
def get_all_sds():
    #read in phot
    phot=Table(fits.open('/volumes/LaCie/zhang/parallaxes.fit')[1].data)
    df=phot.to_pandas()
    df['names']=df.Name.apply(lambda x: x.split(' ')[1].split('\r')[0])
    
    #read in phot
    par=Table(fits.open('/volumes/LaCie/zhang/parallaxes.fit')[1].data)
    df1=par.to_pandas()
    df1['names']=df1.Name.apply(lambda x: x.split(' ')[1].split('\r')[0])
    
    #read in Gaia
    g=Table(fits.open('/volumes/LaCie/zhang/L_sds_Gaia_properties.fit')[1].data)
    df2=g.to_pandas()
    df2['names']=df2.Name.apply(lambda x: x.split(' ')[1].split('\r')[0])
    
    #read in Gaia
    g=Table(fits.open('/volumes/LaCie/zhang/msds_properties.fit')[1].data)
    df3=g.to_pandas()
    df3['names']=df3.Name.apply(lambda x: x.split(' ')[1].split('\r')[0])
    
    
    #read in Gaia
    g=Table(fits.open('/volumes/LaCie/zhang/properties.fit')[1].data)
    df4=g.to_pandas()
    df4['names']=df4.Name.apply(lambda x: x.split(' ')[1].split('\r')[0])
    

    return df.merge(df1, how='left', on='names').merge(df2,  how='left', on='names').merge(df3, how='left', on='names').merge(df4,  on='names', how='left')

In [20]:
df=get_all_sds().astype(str)

In [22]:
df

,recno_x,SName_x,pmRA_x,e_pmRA_x,pmDE_x,e_pmDE_x,Dist_x,E_Dist_x,e_dist_lc_x,Vtan_x,...,Jmag_y,e_Jmag_y,Hmag_y,e_Hmag_y,l_Kmag,Kmag_y,e_Kmag_y,Ref,_RA_y,_DE_y
0,1,UL0118+03,14,12,-80,12,209,20,18,80,...,18.18,0.05,17.73,0.07,,17.56,0.11,26,19.60371,3.69178
1,2,UL0212+06,2,4,-429,5,139,13,12,282,...,17.43,0.03,17.06,0.03,,16.78,0.05,26,33.24196,6.68822
2,3,UL0238+05,186,18,-145,15,95,9,8,106,...,16.43,0.01,16.02,0.02,,15.59,0.02,26,39.513,5.75725
3,4,UL0753+20,-36,1,-191,1,74,7,6,68,...,15.87,0.01,15.48,0.01,,15.09,0.01,26,118.39679,20.10622
4,5,UL0822+04,35,7,-154,5,92,9,8,69,...,16.29,0.01,15.96,0.02,,15.53,0.02,26,125.52754,4.68383
5,6,UL1114+07,-17,9,-306,6,211,20,18,306,...,17.59,0.03,17.26,0.04,,17.12,0.07,26,168.62308,7.46931
6,7,UL1231+01,-225,6,22,4,164,16,14,176,...,17.54,0.03,17.21,0.04,,16.78,0.05,26,187.92912,1.84594
7,8,UL1241-00,-61,9,-45,5,245,23,21,89,...,18.46,0.1,18.08,0.12,,18.04,0.2,26,190.26979,-0.09206
8,9,UL1252+09,-299,7,6,6,117,11,10,166,...,16.87,0.02,16.47,0.02,,16.05,0.03,26,193.11092,9.48892
9,10,UL1307+15,-391,14,-124,13,86,8,8,168,...,18.14,0.04,17.53,0.07,,17.24,0.09,26,196.79258,15.18428


In [23]:
df.to_csv('/volumes/LaCie/zhang/combined.csv')

In [ ]:
phot=Table(fits.open('/volumes/LaCie/zhang/parallaxes.fit')[1].data)

In [ ]:
phot

In [ ]:
data={'spectra': spectra, 'spt': sp_types, 'name': sp_designations, 'photometry': df}

In [ ]:
#data['spectra']
np.save('/Volumes/LaCie/popsimsdata/zhang_subdwarfs_templates.npy', [data], allow_pickle=True)

In [ ]:
df.columns